In [1]:
import os
%pwd

'd:\\Krish Naik Code Practice\\end-to-end-text-summarizer\\research'

In [2]:
# We make sure that the current working directory is the root of the repository.

os.chdir("../")
%pwd

'd:\\Krish Naik Code Practice\\end-to-end-text-summarizer'

In [13]:
# Workflow Step 3: updating entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
# Workflow  Step 4: updating configuartion manager
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, # defined in constants' constructor
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
# Workflow Step 5: update components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Download data from source URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    # Extract zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
# Workflow Step 6:Creating pipeline
try:
    # initialize the configuration manager class
    config = ConfigurationManager() 

    # get the data ingestion config from the class
    data_ingestion_config = config.get_data_ingestion_config()
    
    # call the data ingestion components
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # call the two methods: download_file() and extract_zip_file()
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-08 21:00:00,138: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-08 21:00:00,142: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-08 21:00:00,145: INFO: common: created directory at: artifacts]
[2024-02-08 21:00:00,148: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-08 21:00:02,393: INFO: 4192803348: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6F64:3257:92E6E3:B26804:65C5871F
Accept-Ranges: bytes
Date: Fri, 09 Feb 2024 02:00:01 GMT
Via: 1.1 varnish
X-Served-By: cache-iad-kiad7000161-IAD
X-Cac